In [167]:
%pip install plotly
%pip install chart-studio
%pip install -U scikit-learn
%pip install jupyterlab "ipywidgets>=7.5"

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [168]:
#import de bibliotecas
import pandas as pd
import seaborn as sns
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import statsmodels.api as sm
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score



In [169]:
#leitura de arquivo
df = pd.read_csv("material/MKT.csv")
df.head()

,youtube,facebook,newspaper,sales
0,84.72,19.20,48.96,12.60
1,351.48,33.96,51.84,25.68
2,135.48,20.88,46.32,14.28
3,116.64,1.80,36.00,11.52
4,318.72,24.00,0.36,20.88


In [170]:
#entendendo os dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   youtube    171 non-null    float64
 1   facebook   171 non-null    float64
 2   newspaper  171 non-null    float64
 3   sales      171 non-null    float64
dtypes: float64(4)
memory usage: 5.5 KB


Base de dados sem colunas com campos nulos, todas as colunas são do tipo float

In [171]:
df.describe()

,youtube,facebook,newspaper,sales
count,171.000000,171.000000,171.000000,171.000000
mean,178.021053,27.671579,35.240000,16.922807
std,102.449597,17.913532,24.902918,6.314608
min,0.840000,0.000000,0.360000,1.920000
25%,91.080000,11.700000,13.740000,12.540000
50%,179.760000,26.760000,31.080000,15.480000
75%,262.980000,43.680000,50.880000,20.820000
max,355.680000,59.520000,121.080000,32.400000


Rodando a função describe, podemos analizar que o valor médio e o valor máximo da 0% dos dados estão próximos

e que para a coluna "newspaper" o maior valor está muito distante da média para esta coluna



In [172]:
# Criar uma figura com subplots
fig = make_subplots(rows=3, cols=1)

# Adicionar o primeiro gráfico (YouTube)
fig.add_trace(
    go.Box(x=df["youtube"], line_color='blue'),
    row=1, col=1
)

# Adicionar o segundo gráfico (Facebook)
fig.add_trace(
    go.Box(x=df["facebook"], line_color='green'),
    row=2, col=1
)

# Adicionar o terceiro gráfico (Newspaper)
fig.add_trace(
    go.Box(x=df["newspaper"], line_color='red'),
    row=3, col=1
)

# Atualizar layout, se necessário
fig.update_layout(height=600, width=800, title_text="Distribuição de Dados por Plataforma",template = 'plotly_dark')
fig.show()

Como foi percebido no describe, a categoria 'newspaper' não está com uma distribuição normal

In [173]:

# Criar a matriz de dispersão (pair plot)
fig = px.scatter_matrix(df, dimensions=['youtube', 'facebook', 'newspaper'], color='sales', template = 'plotly_dark')

# Mostrar o gráfico
fig.show()

Com o Gráfico acima conseguimos chegar a conclusão de que a melhor situação é um investimento conjunto de Facebook com o Youtube, embora o youtube sozinho já mostra melhores resultado do que as outras duas mídias isoladas.

In [174]:
# Criar subplots
fig = make_subplots(rows=1, cols=3, shared_yaxes=True,
                    subplot_titles=('YouTube vs Sales', 'Facebook vs Sales', 'Newspaper vs Sales'))

# Adicionar gráficos de dispersão
fig.add_trace(go.Scatter(x=df['youtube'], y=df['sales'], mode='markers', name='YouTube vs Sales'), row=1, col=1)
fig.add_trace(go.Scatter(x=df['facebook'], y=df['sales'], mode='markers', name='Facebook vs Sales'), row=1, col=2)
fig.add_trace(go.Scatter(x=df['newspaper'], y=df['sales'], mode='markers', name='Newspaper vs Sales'), row=1, col=3)

# Atualizar layout
fig.update_layout(height=400, width=1200, title_text="Relação entre mídia e vendas",template = 'plotly_dark')

# Mostrar o gráfico
fig.show()

O gráfico mostra que existe uma relação linear nas categorias 'youtube' e 'facebook' com o 'sales'
 A maior linearidade se dá com a relação de vendas e Youtube e a menor linearidade se da entre Newspaper e vendas

In [175]:
df.corr()

,youtube,facebook,newspaper,sales
youtube,1.000000,0.086538,0.110759,0.782030
facebook,0.086538,1.000000,0.293425,0.602918
newspaper,0.110759,0.293425,1.000000,0.254987
sales,0.782030,0.602918,0.254987,1.000000


In [176]:
# Calcular a matriz de correlação
corr_matrix = df.corr().values

# Obter os nomes das colunas
columns = df.columns

# Personalizando a cor da escala.
custom_colorscale = [
    [0, 'rgb(0,0,0)'], # Branco no valor 0.5
    [0.5, 'rgb(0,0,180)'], # Branco no valor 0.5
    [1, 'rgb(180,0,0)']     # Vermelho no valor 1
]

# Criar o heatmap
heatmap = go.Figure(data=go.Heatmap(
                   z=corr_matrix,
                   x=columns,
                   y=columns,
                   colorscale=custom_colorscale,
                   zmin=-1, zmax=1))

# Adicionar anotações no heatmap
annotations = []
for i in range(len(columns)):
    for j in range(len(columns)):
        annotations.append(
            dict(
                x=columns[j],
                y=columns[i],
                text=str(np.round(corr_matrix[i][j], 2)),
                showarrow=False,
                font=dict(color="black" if abs(corr_matrix[i][j]) < 0.5 else "gray")
            )
        )

heatmap.update_layout(
    title='Matriz de Correlação',
    annotations=annotations,
    xaxis=dict(tickmode='array', tickvals=list(range(len(columns))), ticktext=columns),
    yaxis=dict(tickmode='array', tickvals=list(range(len(columns))), ticktext=columns),
    template = 'plotly_dark'
)

# Mostrar o gráfico
heatmap.show()

A relação entre vendas e divulgação no youtube é de 78%, bem forte. Já a newspaper é de 25%

In [177]:
fig = px.scatter(df, x='sales', y='youtube', trendline='ols',
                 title='Reta de Regressão - Vendas x YouTube',
                 labels={'sales': 'Vendas', 'youtube': 'YouTube'}, 
                 template = 'plotly_dark',
                 color_discrete_sequence=['red'])

# Ajustar o layout
fig.update_layout(
    title={'text': 'Reta de Regressão - Vendas x YouTube', 'x':0.5, 'xanchor': 'center'},
    xaxis_title='Vendas',
    yaxis_title='YouTube'
)

# Mostrar o gráfico
fig.show()

Execução de um grafico isoladao de regreção linear com predição.

In [178]:

# Adicionar constante para o modelo (intercepto)
X = sm.add_constant(df['sales'])
y = df['youtube']

# Ajustar o modelo de regressão linear
model = sm.OLS(y, X).fit()

# Obter os valores preditos
df['predicted'] = model.predict(X)

# Obter o intervalo de confiança
predictions = model.get_prediction(X)
confidence_intervals = predictions.conf_int()

# Criar o gráfico de dispersão com Plotly
fig = px.scatter(df, x='sales', y='youtube', title='Reta de Regressão - Vendas x YouTube', labels={'sales': 'Vendas', 'youtube': 'YouTube'}, color_discrete_sequence=['red'])

# Adicionar a linha de regressão
fig.add_trace(go.Scatter(x=df['sales'], y=df['predicted'], mode='lines', name='Regressão Linear', line=dict(color='blue')))

# Adicionar o intervalo de confiança
fig.add_trace(go.Scatter(
    x=df['sales'], y=confidence_intervals[:, 0], 
    mode='lines', 
    name='Intervalo de Confiança Inferior', 
    line=dict(color='rgba(255,0,255,0.3)'), 
    fill=None
))
fig.add_trace(go.Scatter(
    x=df['sales'], y=confidence_intervals[:, 1], 
    mode='lines', 
    name='Intervalo de Confiança Superior', 
    line=dict(color='rgba(200,255,200,0.3)'), 
    fill='tonexty', 
    fillcolor='rgba(200,255,200,0.3)'
))

# Ajustar o layout do gráfico
fig.update_layout(
    title={'text': 'Reta de Regressão - Vendas x YouTube', 'x':0.5, 'xanchor': 'center'},
    xaxis_title='Vendas',
    yaxis_title='YouTube',
    template = 'plotly_dark',
)

# Mostrar o gráfico
fig.show()

Execução de subplots com 3 graficos de regreção linear com predição para as três midias disponíveis

In [179]:
# Função para criar modelos de regressão e obter os intervalos de confiança
def regression_plot(df, x_col, y_col):
    X = sm.add_constant(df[x_col])
    y = df[y_col]
    model = sm.OLS(y, X).fit()
    df['predicted'] = model.predict(X)
    predictions = model.get_prediction(X)
    confidence_intervals = predictions.conf_int()
    return df, confidence_intervals

# Criar subplots
fig = make_subplots(rows=1, cols=3, subplot_titles=('YouTube vs Sales', 'Facebook vs Sales', 'Newspaper vs Sales'))

# Adicionar gráficos para YouTube
df, ci = regression_plot(df, 'sales', 'youtube')
fig.add_trace(go.Scatter(x=df['sales'], y=df['youtube'], mode='markers', name='YouTube', marker=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=df['sales'], y=df['predicted'], mode='lines', name='Regressão Linear YouTube', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=df['sales'], y=ci[:, 0], mode='lines', name='Intervalo Inferior YouTube', line=dict(color='rgba(255,0,255,0.3)'), showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=df['sales'], y=ci[:, 1], mode='lines', name='Intervalo Superior YouTube', line=dict(color='rgba(200,255,200,0.3)'), fill='tonexty', fillcolor='rgba(173,216,230,0.2)', showlegend=False), row=1, col=1)

# Adicionar gráficos para Facebook
df, ci = regression_plot(df, 'sales', 'facebook')
fig.add_trace(go.Scatter(x=df['sales'], y=df['facebook'], mode='markers', name='Facebook', marker=dict(color='red')), row=1, col=2)
fig.add_trace(go.Scatter(x=df['sales'], y=df['predicted'], mode='lines', name='Regressão Linear Facebook', line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=df['sales'], y=ci[:, 0], mode='lines', name='Intervalo Inferior Facebook', line=dict(color='rgba(255,0,255,0.3)'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=df['sales'], y=ci[:, 1], mode='lines', name='Intervalo Superior Facebook', line=dict(color='rgba(200,255,200,0.3)'), fill='tonexty', fillcolor='rgba(173,216,230,0.2)', showlegend=False), row=1, col=2)

# Adicionar gráficos para Newspaper
df, ci = regression_plot(df, 'sales', 'newspaper')
fig.add_trace(go.Scatter(x=df['sales'], y=df['newspaper'], mode='markers', name='Newspaper', marker=dict(color='red')), row=1, col=3)
fig.add_trace(go.Scatter(x=df['sales'], y=df['predicted'], mode='lines', name='Regressão Linear Newspaper', line=dict(color='blue')), row=1, col=3)
fig.add_trace(go.Scatter(x=df['sales'], y=ci[:, 0], mode='lines', name='Intervalo Inferior Newspaper', line=dict(color='rgba(255,0,255,0.3)'), showlegend=False), row=1, col=3)
fig.add_trace(go.Scatter(x=df['sales'], y=ci[:, 1], mode='lines', name='Intervalo Superior Newspaper', line=dict(color='rgba(200,255,200,0.3)'), fill='tonexty', fillcolor='rgba(173,216,230,0.2)', showlegend=False), row=1, col=3)

# Ajustar o layout do gráfico
fig.update_layout(
    height=600,
    width=1500,
    title={'text': 'Regressão Linear para YouTube, Facebook e Newspaper', 'x':0.5, 'xanchor': 'center'},
    xaxis_title='Vendas',
    yaxis_title='Valor',
    template = 'plotly_dark',
)

# Mostrar o gráfico
fig.show()

Reflexão:

Os dados do YouTube apresentam a maior linearidade e previsibilidade, indicando uma forte correlação linear entre as vendas e os investimentos em publicidade na plataforma. Essa relação linear facilita a previsão de vendas com base nos valores de investimento no YouTube.

Por outro lado, os dados de Newspaper mostram pouca ou nenhuma linearidade, tornando a predição mais difícil e imprecisa. A falta de correlação clara sugere que os investimentos em publicidade em jornais não têm um impacto direto e previsível sobre as vendas.

In [180]:
df.columns

Index(['youtube', 'facebook', 'newspaper', 'sales', 'predicted'], dtype='object')

In [181]:
x=df[['youtube', 'facebook']]
y=df['sales']

Os dados de newspapper foram descartados dada a falta de previsibilidade.

#Treinamento

In [182]:
#Dividindo os dados para treinamento
x_train, x_test, y_train, y_test = train_test_split(x, y , train_size=0.7, test_size = 0.3, random_state=42)

In [183]:
lm = LinearRegression()

In [184]:
#executando o treino
lm.fit(x_train, y_train)

LinearRegression()

In [185]:
#gerar a predição
y_pred = lm.predict(x_test)

In [186]:
#r quadrado
r=r2_score(y_test, y_pred)

In [187]:
print("r_quadrado: ", r)

r_quadrado:  0.8824796927714542


In [188]:
#expectativa de venda atravez do investimento
youtube = 150.0
facebook = 50.0

entrada = pd.DataFrame([[youtube, facebook]], columns=['youtube', 'facebook'])
print(lm.predict(entrada)[0])
lm.predict(entrada)[0]

19.84633834051536


19.84633834051536